<a href="https://colab.research.google.com/github/atsiniloiv/ucmnticmaster-deaplearning-exam/blob/main/Examen_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dado que el entrenamiento de redes neuronales es una tarea  muy costosa, **se recomienda ejecutar el notebooks en [Google Colab](https://colab.research.google.com)**, por supuesto también se puede ejecutar en local.

Al entrar en [Google Colab](https://colab.research.google.com) bastará con hacer click en `upload` y subir este notebook. No olvide luego descargarlo en `File->Download .ipynb`

**El examen deberá ser entregado con las celdas ejecutadas, si alguna celda no está ejecutadas no se contará.**

El examen se divide en dos partes, con la puntuación que se indica a continuación. La puntuación máxima será 10.

- [Actividad 1: Redes Densas](#actividad_1): 5 pts
    - Correcta normalización: máximo de 0.25 pts
    - [Cuestión 1](#1.1): 1.5 pt
    - [Cuestión 2](#1.2): 1.5 pt
    - [Cuestión 3](#1.3): 0.5 pts
    - [Cuestión 4](#1.4): 0.25 pts
    - [Cuestión 5](#1.5): 0.25 pts
    - [Cuestión 6](#1.6): 0.25 pts
    - [Cuestión 7](#1.7): 0.25 pts
    - [Cuestión 8](#1.8): 0.25 pts


- [Actividad 2: Redes Convolucionales](#actividad_2): 5 pts
    - [Cuestión 1](#2.1): 2.5 pt
    - [Cuestión 2](#2.2): 1 pt
    - [Cuestión 3](#2.3): 0.5 pts
    - [Cuestión 4](#2.4): 0.5 pts
    - [Cuestión 5](#2.5): 0.5 pts
    

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
tf.random.set_seed(0)

<a name='actividad_1'></a>
# Actividad 1: Redes Densas

Para esta primera actividad vamos a utilizar el [wine quality dataset](https://archive.ics.uci.edu/ml/datasets/wine+quality). Con el que trataremos de predecir la calidad del vino.

La calidad del vino puede tomar valores decimales (por ejemplo 7.25), independientemente de que en el dataset de entrenamiento sean números enteros. Por lo tanto, el problema es una `regresión`.

**Puntuación**:

Normalizar las features correctamente (x_train, x_test): 0.25 pts , se pueden normalizar con el [Normalization layer](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Normalization) de Keras.


- Correcta normalización: máximo de 0.25 pts
- [Cuestión 1](#1.1): 1 pt
- [Cuestión 2](#1.2): 1 pt
- [Cuestión 3](#1.3): 0.5 pts
- [Cuestión 4](#1.4): 0.25 pts
- [Cuestión 5](#1.5): 0.25 pts
- [Cuestión 6](#1.6): 0.25 pts
- [Cuestión 7](#1.7): 0.25 pts
- [Cuestión 8](#1.8): 0.25 pts



In [2]:
# Descargar los datos con pandas
df_red = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv',
    sep=';'
)
df_white = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv',
    sep=';'
)
df = pd.concat([df_red, df_white])

df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
feature_names = [
    'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides',
    'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol'
]

# separar features y target
y = df.pop('quality').values
X = df.copy().values

In [4]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

print('x_train, y_train shapes:', x_train.shape, y_train.shape)
print('x_test, y_test shapes:', x_test.shape, y_test.shape)
print('Some qualities: ', y_train[:5])

x_train, y_train shapes: (4872, 11) (4872,)
x_test, y_test shapes: (1625, 11) (1625,)
Some qualities:  [6 7 8 5 6]


In [5]:
## Normalizar las features

from tensorflow.keras.layers import Normalization
normalizer = keras.layers.Normalization()
normalizer.adapt(x_train) # voy a normalizar usando los parámetros aprendidos de x_train
x_train_norm = normalizer(x_train)
x_test_norm = normalizer(x_test) # x_test_norm puede no tener media 0 y std 1 exactos
# ya que está normalizado usando los parámetros aprendidos de x_train

x_train_norm[1,:] # visualizo los valores de las variable normalizadas en datos de entrenamiento

<tf.Tensor: shape=(11,), dtype=float32, numpy=
array([-0.7853838 , -1.0907232 ,  0.07390133, -0.9196466 ,  0.0222696 ,
       -0.5301523 , -0.596653  , -1.2502729 ,  0.6399148 , -0.47886032,
        0.34073704], dtype=float32)>

<a name='1.1'></a>
## Cuestión 1: Cree un modelo secuencial que contenga 4 capas ocultas(hidden layers), con más de 60 neuronas  por capa, sin regularización y obtenga los resultados.

Puntuación:
- Obtener el modelo correcto: 0.8 pts
- Compilar el modelo: 0.1pts
- Acertar con la función de pérdida: 0.1 pts

In [6]:
# Definir el modelo
model_1 = keras.Sequential([
    keras.Input(shape=(11, )),
    layers.Dense(65, activation='relu', name='layer1'), # uso relu como función de activación en las capas ocultas
    layers.Dense(65, activation='relu', name='layer2'),
    layers.Dense(65, activation='relu', name='layer3'),
    layers.Dense(65, activation='relu', name='layer4'), # 4 capas ocultas de 65 neuronas cada una
    layers.Dense(1, activation='linear', name='layerout') # uso linear para tener una salida libre (float)
])

model_1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ layer1 (Dense)                  │ (None, 65)             │           780 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer2 (Dense)                  │ (None, 65)             │         4,290 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer3 (Dense)                  │ (None, 65)             │         4,290 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer4 (Dense)                  │ (None, 65)             │         4,290 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layerout (Dense)                │ (None, 1)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,716 (53.58 KB)

 Trainable params: 13,716 (53.58 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Compilar el modelo

model_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae'] # por ver también esta métrica, además del mean squared error
)

In [9]:
# No modifique el código
model_1.fit(x_train_norm,
          y_train,
          epochs=200,
          batch_size=32,
          validation_split=0.2,
          verbose=1)

Epoch 1/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 15.7930 - mae: 3.2834 - val_loss: 1.8114 - val_mae: 0.9559
Epoch 2/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1.3203 - mae: 0.8990 - val_loss: 1.1282 - val_mae: 0.7689
Epoch 3/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8462 - mae: 0.7162 - val_loss: 0.7911 - val_mae: 0.6485
Epoch 4/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6258 - mae: 0.6104 - val_loss: 0.6544 - val_mae: 0.6012
Epoch 5/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5461 - mae: 0.5677 - val_loss: 0.6076 - val_mae: 0.5851
Epoch 6/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.5131 - mae: 0.5513 - val_loss: 0.5852 - val_mae: 0.5773
Epoch 7/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.4941 - mae: 0.5420 - val_loss: 0.5762 - val_mae: 0.5765
Epoch 8/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4809 - mae: 0.5350 - val_loss: 0.5686 - val_mae: 0.5745
Epoch 9/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3m

In [10]:
# No modifique el código
results_1 = model_1.evaluate(x_test_norm, y_test, verbose=1)
print('Test Loss: {}'.format(results_1))

51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.7782 - mae: 0.6688
Test Loss: [0.7908891439437866, 0.678528904914856]


<a name='1.2'></a>
## Cuestión 2: Utilice el mismo modelo de la cuestión anterior pero añadiendo al menos dos técnicas distinas de regularización.

Ejemplos de regularización: [Prevent_Overfitting.ipynb](https://github.com/ezponda/intro_deep_learning/blob/main/class/Fundamentals/Prevent_Overfitting.ipynb)

Puntuación:

- Obtener el modelo con la regularización: 0.8 pts
- Obtener un `test loss` inferior al anterior: 0.2 pts


In [6]:
# Una manera de regularizar el modelo es hacerlo más simple, con un menor número de parámetros.
# Por lo tanto eliminé una de las capas ocultas y dejé el modelo en 3.
# También incluyo un Dropout layer después de cada capa oculta para
# 'apagar' aleatoriamente un 30% de las neuronas durante el entrenamiento.
model_2 = keras.Sequential([
    keras.Input(shape=(11, )),
    layers.Dense(65, activation='relu', name='layer1'),
    layers.Dropout(0.3, name='dropout_layer1'),
    layers.Dense(65, activation='relu', name='layer2'),
    layers.Dropout(0.3, name='dropout_layer2'),
    layers.Dense(65, activation='relu', name='layer3'),
    layers.Dropout(0.3, name='dropout_layer3'),
    layers.Dense(1, activation='linear', name='layerout')
])

model_2.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ layer1 (Dense)                  │ (None, 65)             │           780 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_layer1 (Dropout)        │ (None, 65)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer2 (Dense)                  │ (None, 65)             │         4,290 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_layer2 (Dropout)        │ (None, 65)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer3 (Dense)                  │ (None, 65)             │         4,290 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_layer3 (Dropout)        │ (None, 65)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layerout (Dense)                │ (None, 1)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,426 (36.82 KB)

 Trainable params: 9,426 (36.82 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Compilación del modelo
model_2.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae'] # por ver también esta métrica, además del mean squared error
)

In [8]:
# Otro método es reducir el batch size de manera que el modelo actualiza los pesos
# cada 32 muestras y no después de cada una.
batch_size=32

In [9]:
# No modifique el código
model_2.fit(x_train_norm,
          y_train,
          epochs=200,
          batch_size=batch_size,
          validation_split=0.2,
          verbose=1)

Epoch 1/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - loss: 15.7204 - mae: 3.3952 - val_loss: 2.3221 - val_mae: 1.2023
Epoch 2/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.2773 - mae: 1.4543 - val_loss: 1.4894 - val_mae: 0.9401
Epoch 3/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.4990 - mae: 1.2624 - val_loss: 1.3162 - val_mae: 0.9044
Epoch 4/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.9813 - mae: 1.1150 - val_loss: 0.9970 - val_mae: 0.7755
Epoch 5/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.6267 - mae: 1.0183 - val_loss: 0.9183 - val_mae: 0.7500
Epoch 6/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.4643 - mae: 0.9551 - val_loss: 0.8722 - val_mae: 0.7326
Epoch 7/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.4216 - mae: 0.9341 - val_loss: 0.8352 - val_mae: 0.7145
Epoch 8/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.2781 - mae: 0.8931 - val_loss: 0.7508 - val_mae: 0.6720
Epoch 9/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3m

In [10]:
# No modifique el código
results_2 = model_2.evaluate(x_test_norm, y_test, verbose=1)
print('Test Loss: {}'.format(results_2))

51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4456 - mae: 0.5202
Test Loss: [0.4689842164516449, 0.5311887860298157]


<a name='1.3'></a>
## Cuestión 3: Utilice el mismo modelo de la cuestión anterior pero añadiendo un callback de early stopping.

In [ ]:
model = tf.keras.models.Sequential()
# Código aquí
...

In [ ]:
# Compilación del modelo
# Código aquí
model.compile(...)

In [ ]:
## definir el early stopping callback
# Código aquí
...
model.fit(x_train,
          y_train,
          epochs=200,
          batch_size=32,
          validation_split=0.2,
          verbose=1,
          callbacks=[...]) # Código aquí

In [ ]:
# No modifique el código
results = model.evaluate(x_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

<a name='1.4'></a>
## Cuestión 4: ¿Podría haberse usado otra función de activación de la neurona de salida? En caso afirmativo especifíquela.

<a name='1.5'></a>
## Cuestión 5:  ¿Qué es lo que una neurona calcula?

**a)** Una función de activación seguida de una suma ponderada  de las entradas.

**b)** Una suma ponderada  de las entradas seguida de una función de activación.

**c)** Una función de pérdida, definida sobre el target.

**d)** Ninguna  de las anteriores es correcta


<a name='1.6'></a>
## Cuestión 6:  ¿Cuál de estas funciones de activación no debería usarse en una capa oculta (hidden layer)?

**a)** `sigmoid`

**b)** `tanh`

**c)** `relu`

**d)** `linear`


<a name='1.7'></a>
## Cuestión 7:  ¿Cuál de estas técnicas es efectiva para combatir el overfitting en una red con varias capas ocultas? Ponga todas las que lo sean.

**a)** Dropout

**b)** Regularización L2.

**c)** Aumentar el tamaño del test set.

**d)** Aumentar el tamaño del validation set.

**e)** Reducir el número de capas de la red.

**f)** Data augmentation.

<a name='1.8'></a>
## Cuestión 8:  Supongamos que queremos entrenar una red para un problema de clasificación de imágenes con las siguientes clases: {'perro','gato','persona'}. ¿Cuántas neuronas y que función de activación debería tener la capa de salida? ¿Qué función de pérdida (loss function) debería usarse?


<a name='actividad_2'></a>
# Actividad 2: Redes Convolucionales

Vamos a usar el dataset [cifar-10](https://www.cs.toronto.edu/~kriz/cifar.html), que son 60000 imágenes de 32x32 a color  con 10 clases diferentes. Para realizar mejor la práctica puede consultar [Introduction_to_CNN.ipynb](https://github.com/ezponda/intro_deep_learning/blob/main/class/CNN/Introduction_to_CNN.ipynb).



**Puntuación**:

- [Cuestión 1](#2.1): 2.5 pt
- [Cuestión 2](#2.2): 1 pt
- [Cuestión 3](#2.3): 0.5 pts
- [Cuestión 4](#2.4): 0.5 pts
- [Cuestión 5](#2.5): 0.5 pts

Puede normalizar las imágenes al principio o usar la capa [Rescaling](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Rescaling):

```python
tf.keras.layers.experimental.preprocessing.Rescaling(
    scale, offset=0.0, name=None, **kwargs
)
```

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
y_train = y_train.flatten()
y_test = y_test.flatten()

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i])
    plt.xlabel(class_names[y_train[i]])
plt.show()

In [ ]:
print('x_train, y_train shapes:', x_train.shape, y_train.shape)
print('x_test, y_test shapes:', x_test.shape, y_test.shape)

<a name='2.1'></a>
## Cuestión 1: Cree una red convolucional con la API funcional con al menos dos capas convolucionales y al menos dos capas de pooling. Debe obtener un `Test accuracy > 0.68`

In [ ]:
inputs = tf.keras.Input(shape=..., name='input')
# reescaling = ...

# Convolution + pooling layers
...

# Flattening
...

# Fully-connected
outputs = layers.Dense(...)

model = keras.Model(inputs=..., outputs=...)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=25, batch_size=64,
                    validation_split=0.15)

In [ ]:
results = model.evaluate(x_test, y_test, verbose=0, batch_size=1000)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

<a name='2.2'></a>
## Cuestión 2: Cree el mismo  modelo de manera secuencial. No es necesario compilar ni entrenar el modelo

In [ ]:
model_seq = tf.keras.models.Sequential()
# Código aquí
...

<a name='2.3'></a>
## Cuestión 3: Si tenenemos una  una imagen de entrada de 300 x 300 a color (RGB) y queremos usar una red densa. Si la primera capa oculta tiene 100 neuronas, ¿Cuántos parámetros tendrá esa capa (sin incluir el bias) ?


<a name='2.4'></a>
## Cuestión 4   Ponga  las verdaderas ventajas de las redes convolucionales respecto a las densas

**a)** Reducen el número total de parámetros, reduciendo así el overfitting.

**b)** Permiten utilizar una misma 'función'  en varias localizaciones de la imagen de entrada, en lugar de aprender una función diferente para cada pixel.

**c)** Permiten el uso del transfer learning.

**d)** Generalmente son menos profundas, lo que facilita su entrenamiento.



<a name='2.5'></a>
## Cuestión 5: Para el procesamiento de series temporales las redes convolucionales no son efectivas, habrá que usar redes recurrentes.

- **Verdadero**
- **Falso**